In [1]:
import tensorflow as tf 
import pandas as pd
import numpy as np

from pprint import pformat

In [2]:
with tf.io.gfile.GFile("gs://vit_models/augreg/index.csv") as f:
    df = pd.read_csv(f)

df.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
0,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,imagenet2012,0.030,20000,384,0.755698,0.72874,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
1,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,imagenet2012,0.010,20000,384,0.754605,0.72412,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
2,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.030,10000,384,0.836000,0.83380,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
3,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.010,10000,384,0.835000,0.83040,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
4,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.003,10000,384,0.800000,0.79620,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...


In [3]:
b16s = df.query(
    'ds=="i21k" & adapt_resolution==224 & adapt_ds=="imagenet2012" & name=="B/16"'
).sort_values("adapt_final_test", ascending=False)
b16s.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
24854,B/16,i21k,300.0,0.001,medium2,0.03,0.0,0.0,0.504258,0.503623,...,imagenet2012,0.03,20000,224,0.882454,0.84018,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...
24039,B/16,i21k,300.0,0.001,medium1,0.10,0.0,0.0,0.506260,0.506055,...,imagenet2012,0.01,20000,224,0.875507,0.83962,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...,B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...
24922,B/16,i21k,300.0,0.001,strong1,0.03,0.0,0.0,0.503555,0.503330,...,imagenet2012,0.03,20000,224,0.877849,0.83920,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_strong1-wd_0.03-d...,B_16-i21k-300ep-lr_0.001-aug_strong1-wd_0.03-d...
23971,B/16,i21k,300.0,0.001,light1,0.10,0.0,0.0,0.504121,0.503955,...,imagenet2012,0.01,20000,224,0.882532,0.83824,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_...,B_16-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_...
24107,B/16,i21k,300.0,0.001,medium2,0.10,0.0,0.0,0.504111,0.503887,...,imagenet2012,0.01,20000,224,0.872854,0.83796,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do...,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do...


In [4]:
best_b16_i1k_checkpoint = str(b16s.iloc[0]["adapt_filename"])
b16s.iloc[0]["adapt_filename"], b16s.iloc[0]["adapt_final_test"]

('B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224',
 0.8401799999999999)

In [5]:
filename = best_b16_i1k_checkpoint

path = f"gs://vit_models/augreg/{filename}.npz"

print(f"{tf.io.gfile.stat(path).length / 1024 / 1024:.1f} MiB - {path}")

330.3 MiB - gs://vit_models/augreg/B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz


In [6]:
# !gsutil cp {path} .
local_path = path.split("//")[-1].split("/")[-1]
local_path

'B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'

In [7]:
with open(local_path, "rb") as f:
    params_tf = np.load(f)
    params_tf = dict(zip(params_tf.keys(), params_tf.values()))

print(pformat(list(params_tf.keys())))

['Transformer/encoder_norm/bias',
 'Transformer/encoder_norm/scale',
 'Transformer/encoderblock_0/LayerNorm_0/bias',
 'Transformer/encoderblock_0/LayerNorm_0/scale',
 'Transformer/encoderblock_0/LayerNorm_2/bias',
 'Transformer/encoderblock_0/LayerNorm_2/scale',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias',
 'Tr

In [8]:
import sys

sys.path.append("..")

from vit.configs import base_config
from vit.layers import mha
from vit.models import ViTClassifierExtended

In [9]:
config = base_config.get_config()
with config.unlocked():
    config.num_classes = 1000

config.to_dict()

{'batch_size': 32,
 'buffer_size': 64,
 'classifier': 'token',
 'dropout_rate': 0.0,
 'image_size': 224,
 'initializer_range': 0.02,
 'input_shape': (224, 224, 3),
 'layer_norm_eps': 1e-06,
 'mlp_units': [3072, 768],
 'num_classes': 1000,
 'num_heads': 12,
 'num_layers': 12,
 'num_patches': 196,
 'patch_size': 16,
 'pos_emb_mode': 'sincos',
 'projection_dim': 768}

In [10]:
vit_b16_model = ViTClassifierExtended(config)
vit_b16_model(tf.ones((1, 224, 224, 3)))[0].shape

2022-03-21 16:54:09.006641: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([1, 1000])

In [12]:
# Projection.

vit_b16_model.layers[0].layers[0].kernel.assign(
    tf.Variable(params_tf["embedding/kernel"])
)
vit_b16_model.layers[0].layers[0].bias.assign(tf.Variable(params_tf["embedding/bias"]))
print(" ")

In [26]:
np.testing.assert_allclose(
    vit_b16_model.layers[0].layers[0].kernel.numpy(), params_tf["embedding/kernel"]
)

In [27]:
np.testing.assert_allclose(
    vit_b16_model.layers[0].layers[0].bias.numpy(), params_tf["embedding/bias"]
)

In [13]:
# Positional embedding.

vit_b16_model.positional_embedding.assign(
    tf.Variable(params_tf["Transformer/posembed_input/pos_embedding"])
)
print(" ")

In [28]:
np.testing.assert_allclose(
    vit_b16_model.positional_embedding.numpy(),
    params_tf["Transformer/posembed_input/pos_embedding"],
)

In [14]:
# Cls token.

vit_b16_model.cls_token.assign(tf.Variable(params_tf["cls"]))
print(" ")

In [29]:
np.testing.assert_allclose(vit_b16_model.cls_token.numpy(), params_tf["cls"])

In [15]:
# Final layer norm layer.
vit_b16_model.layers[-2].gamma.assign(
    tf.Variable(params_tf["Transformer/encoder_norm/scale"])
)
vit_b16_model.layers[-2].beta.assign(
    tf.Variable(params_tf["Transformer/encoder_norm/bias"])
)

print(" ")

In [30]:
np.testing.assert_allclose(
    vit_b16_model.layers[-2].gamma.numpy(), params_tf["Transformer/encoder_norm/scale"]
)

In [31]:
np.testing.assert_allclose(
    vit_b16_model.layers[-2].beta.numpy(), params_tf["Transformer/encoder_norm/bias"]
)

In [16]:
# Head layer.

vit_b16_model.layers[-1].kernel.assign(tf.Variable(params_tf["head/kernel"]))
vit_b16_model.layers[-1].bias.assign(tf.Variable(params_tf["head/bias"]))
print(" ")

In [32]:
np.testing.assert_allclose(
    vit_b16_model.layers[-1].kernel.numpy(), params_tf["head/kernel"]
)

In [34]:
np.testing.assert_allclose(
    vit_b16_model.layers[-1].bias.numpy(), params_tf["head/bias"]
)

```md
 'Transformer/encoderblock_0/LayerNorm_0/bias',
 'Transformer/encoderblock_0/LayerNorm_0/scale',
 'Transformer/encoderblock_0/LayerNorm_2/bias',
 'Transformer/encoderblock_0/LayerNorm_2/scale',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/kernel',
```

In [17]:
def modify_transformer_block(config, tf_block: tf.keras.Model, jax_block_name: str):
    # LayerNorm layers.
    layer_norm_idx = 0
    for layer in tf_block.layers:
        if isinstance(layer, tf.keras.layers.LayerNormalization):
            layer_norm_jax_prefix = (
                f"Transformer/{jax_block_name}/LayerNorm_{layer_norm_idx}"
            )
            layer.gamma.assign(tf.Variable(params_tf[f"{layer_norm_jax_prefix}/scale"]))
            layer.beta.assign(tf.Variable(params_tf[f"{layer_norm_jax_prefix}/bias"]))
            layer_norm_idx += 2

    # FFN layers.
    ffn_layer_idx = 0
    for layer in tf_block.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            dense_layer_jax_prefix = (
                f"Transformer/{jax_block_name}/MlpBlock_3/Dense_{ffn_layer_idx}"
            )
            layer.kernel.assign(
                tf.Variable(params_tf[f"{dense_layer_jax_prefix}/kernel"])
            )
            layer.bias.assign(tf.Variable(params_tf[f"{dense_layer_jax_prefix}/bias"]))
            ffn_layer_idx += 1

    def modify_attention_block(tf_component, jax_component):
        tf_component.kernel.assign(
            tf.Variable(
                params_tf[f"{jax_component}/kernel"].reshape(config.projection_dim, -1)
            )
        )
        tf_component.bias.assign(
            tf.Variable(
                params_tf[f"{jax_component}/bias"].reshape(config.projection_dim)
            )
        )

    # Attention layer.
    for layer in tf_block.layers:
        attn_layer_jax_prefix = (
            "Transformer/encoderblock_0/MultiHeadDotProductAttention_1"
        )
        if isinstance(layer, mha.TFViTAttention):
            # Key
            modify_attention_block(
                layer.self_attention.key, f"{attn_layer_jax_prefix}/key"
            )
            # Query
            modify_attention_block(
                layer.self_attention.query, f"{attn_layer_jax_prefix}/query"
            )
            # Value
            modify_attention_block(
                layer.self_attention.value, f"{attn_layer_jax_prefix}/value"
            )
            # Final dense projection
            layer.dense_output.dense.kernel.assign(
                tf.Variable(
                    params_tf[f"{attn_layer_jax_prefix}/out/kernel"].reshape(
                        -1, config.projection_dim
                    )
                )
            )
            layer.dense_output.dense.bias.assign(
                tf.Variable(params_tf[f"{attn_layer_jax_prefix}/out/bias"])
            )

In [18]:
idx = 0
for layer in vit_b16_model.layers:
    if isinstance(layer, tf.keras.Model) and layer.name != "projection":
        modify_transformer_block(
            config, vit_b16_model.get_layer(layer.name), f"encoderblock_{idx}"
        )
        idx += 1

In [20]:
import requests
from PIL import Image
from io import BytesIO

In [19]:
def preprocess_image(image):
    image = np.array(image)
    image_resized = tf.image.resize(image, (224, 224))
    image_resized = tf.cast(image_resized, tf.float32)
    image_resized = (image_resized - 127.5) / 127.5
    return tf.expand_dims(image_resized, 0).numpy()

def load_image_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image = preprocess_image(image)
    return image

!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt -O ilsvrc2012_wordnet_lemmas.txt

--2022-03-21 16:55:04--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.200.144, 142.250.67.48, 142.250.182.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.200.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0.04s   

2022-03-21 16:55:05 (600 KB/s) - ‘ilsvrc2012_wordnet_lemmas.txt’ saved [21675/21675]



In [21]:
with open("ilsvrc2012_wordnet_lemmas.txt", "r") as f:
    lines = f.readlines()
imagenet_int_to_str = [line.rstrip() for line in lines]

img_url = "https://p0.pikrepo.com/preview/853/907/close-up-photo-of-gray-elephant.jpg"
image = load_image_from_url(img_url)

In [24]:
predictions = vit_b16_model.predict(image)
logits = predictions[0]
predicted_label = imagenet_int_to_str[int(np.argmax(logits))]
expected_label = "Indian_elephant, Elephas_maximus"
assert (
    predicted_label == expected_label
), f"Expected {expected_label} but was {predicted_label}"

AssertionError: Expected Indian_elephant, Elephas_maximus but was lakeside, lakeshore